In [1]:
#!pip install mlflow
#!pip install azureml-core
#!pip install azureml-mlflow




In [1]:
import mlflow
import mlflow.lightgbm


In [2]:
mlflow.set_experiment("CreditCard_Fraud_LightGBM")


2025/10/08 09:54:57 INFO mlflow.tracking.fluent: Experiment with name 'CreditCard_Fraud_LightGBM' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/Mega-Pc/Freelance/CreditCardFraudDetection/mlruns/765412235952787374', creation_time=1759913697492, experiment_id='765412235952787374', last_update_time=1759913697492, lifecycle_stage='active', name='CreditCard_Fraud_LightGBM', tags={}>

In [5]:
import pickle
from sklearn.metrics import roc_auc_score, classification_report
import pandas as pd
model_path = r"C:\Users\Mega-Pc\Freelance\data\lightgbm_model.pkl"

with open(model_path, "rb") as f:
    LightGBM = pickle.load(f)

#Import data
train_df = pd.read_csv('C:/Users/Mega-Pc/Freelance/data/train_df_cleaned_new.csv', delimiter=',', encoding='utf-8',index_col=0)
test_df = pd.read_csv('C:/Users/Mega-Pc/Freelance/data/test_df_cleaned_new.csv', delimiter=',', encoding='utf-8',index_col=0)

#Data splitting
X_train = train_df.drop(columns=['is_fraud'])  
y_train = train_df['is_fraud']  

X_test = test_df.drop(columns=['is_fraud'])  
y_test = test_df['is_fraud']  



In [6]:
y_pred_proba = LightGBM.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= 0.5).astype(int)

auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC = {auc:.4f}")
print(classification_report(y_test, y_pred))


AUC = 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.64      1.00      0.78      2145

    accuracy                           1.00    555719
   macro avg       0.82      1.00      0.89    555719
weighted avg       1.00      1.00      1.00    555719



In [7]:
with mlflow.start_run(run_name="LightGBM_loaded_model"):
    mlflow.log_param("model_type", "LightGBM ")
    mlflow.log_param("data_source", "Credit Card Fraud Kaggle")
    
    mlflow.log_metric("roc_auc", auc)

    mlflow.lightgbm.log_model(LightGBM, artifact_path="model")

    print(f"✅ Modèle LightGBM loggé dans MLflow avec AUC={auc:.4f}")


2025/10/08 10:01:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Modèle LightGBM loggé dans MLflow avec AUC=1.0000


In [8]:
!python -m mlflow ui


^C


In [10]:
test_df

,merchant,amt,first,last,street,city,zip,lat,long,city_pop,...,age_group_senior,amt_category_medium,amt_category_high,trans_date_trans_time_year,trans_date_trans_time_month,trans_date_trans_time_day,trans_date_trans_time_weekday,trans_date_trans_time_hour,dob_weekday,dob_hour
cc_num,,,,,,,,,,,,,,,,,,,,,
60416207185,0.523121,0.605444,0.658824,0.219149,0.994583,0.304245,0.823573,0.663404,0.238664,0.363638,...,False,False,True,0.0,0.0,0.666667,1.000000,0.565217,0.0,0.0
60416207185,0.325145,0.538535,0.658824,0.219149,0.994583,0.304245,0.823573,0.663404,0.238664,0.363638,...,False,False,True,0.0,0.0,0.666667,1.000000,0.695652,0.0,0.0
60416207185,0.148844,0.511837,0.658824,0.219149,0.994583,0.304245,0.823573,0.663404,0.238664,0.363638,...,False,True,False,0.0,0.0,0.700000,0.000000,0.304348,0.0,0.0
60416207185,0.539017,0.554576,0.658824,0.219149,0.994583,0.304245,0.823573,0.663404,0.238664,0.363638,...,False,False,True,0.0,0.0,0.700000,0.000000,0.652174,0.0,0.0
60416207185,0.552023,0.630376,0.658824,0.219149,0.994583,0.304245,0.823573,0.663404,0.238664,0.363638,...,False,False,True,0.0,0.0,0.733333,0.166667,0.521739,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4992346398065154184,0.684971,0.513723,0.085294,0.485106,0.927411,0.553066,0.608915,0.600027,0.604544,0.266659,...,True,True,False,0.0,1.0,1.000000,0.500000,0.086957,0.0,0.0
4992346398065154184,0.093931,0.150035,0.085294,0.485106,0.927411,0.553066,0.608915,0.600027,0.604544,0.266659,...,True,False,False,0.0,1.0,1.000000,0.500000,0.217391,0.0,0.0
4992346398065154184,0.234104,0.567530,0.085294,0.485106,0.927411,0.553066,0.608915,0.600027,0.604544,0.266659,...,True,False,True,0.0,1.0,1.000000,0.500000,0.478261,0.0,0.0


In [17]:
import pickle
import requests
import json
import pandas as pd
import os


model_path = r"C:\Users\Mega-Pc\Freelance\data\lightgbm_model.pkl"
with open(model_path, "rb") as f:
    model = pickle.load(f)


SLACK_WEBHOOK_URL = os.getenv("SLACK_WEBHOOK_URL")

def send_slack_alert(transaction_index, probability, amount=None, user_id=None):
   
    color = "#FF0000" if probability > 0.9 else "#FFA500"  

    blocks = [
        {
            "type": "header",
            "text": {
                "type": "plain_text",
                "text": "🚨 Fraud Alert Detected!",
                "emoji": True
            }
        },
        {
            "type": "section",
            "fields": [
                {"type": "mrkdwn", "text": f"*Transaction Index:*\n`{transaction_index}`"},
                {"type": "mrkdwn", "text": f"*User ID:*\n`{user_id if user_id else 'N/A'}`"},
                {"type": "mrkdwn", "text": f"*Amount:*\n`${amount if amount else 'N/A'}`"},
                {"type": "mrkdwn", "text": f"*Fraud Probability:*\n*{probability:.2f}*"},
            ]
        },
        {
            "type": "divider"
        },
        {
            "type": "context",
            "elements": [
                {
                    "type": "mrkdwn",
                    "text": ":rotating_light: *Immediate review required!*"
                }
            ]
        },
        {
            "type": "actions",
            "elements": [
                {
                    "type": "button",
                    "text": {
                        "type": "plain_text",
                        "text": "View Transaction Details"
                    },
                    "style": "danger",
                    "url": f"https://dashboard.yourcompany.com/transactions/{transaction_index}"
                },
                {
                    "type": "button",
                    "text": {
                        "type": "plain_text",
                        "text": "Mark as Reviewed"
                    },
                    "style": "primary",
                    "url": f"https://dashboard.yourcompany.com/review/{transaction_index}"
                }
            ]
        }
    ]

    payload = {"attachments": [{"color": color, "blocks": blocks}]}

    try:
        response = requests.post(SLACK_WEBHOOK_URL, data=json.dumps(payload),
                                 headers={'Content-Type': 'application/json'})
        if response.status_code == 200:
            print(f"✅ Alert sent for transaction {transaction_index} (prob={probability:.2f})")
        else:
            print(f"❌ Failed to send Slack alert: {response.status_code}, {response.text}")
    except Exception as e:
        print(f"⚠️ Error sending Slack alert: {e}")



pred_probs = model.predict_proba(X_test)[:, 1]
threshold = 0.8

for idx, prob in enumerate(pred_probs):
    if prob > threshold:
        CC_num = X_test.iloc[idx]['cc_num'] if 'cc_num' in X_test.columns else None
        amount = X_test.iloc[idx]['amt'] if 'amt' in X_test.columns else None
        send_slack_alert(idx, prob, amount, user_id)


✅ Alert sent for transaction 1906 (prob=1.00)
✅ Alert sent for transaction 1907 (prob=1.00)
✅ Alert sent for transaction 1908 (prob=1.00)
✅ Alert sent for transaction 1909 (prob=1.00)
✅ Alert sent for transaction 1910 (prob=1.00)
✅ Alert sent for transaction 1911 (prob=1.00)
✅ Alert sent for transaction 1912 (prob=1.00)
✅ Alert sent for transaction 1913 (prob=1.00)
✅ Alert sent for transaction 1922 (prob=0.94)
✅ Alert sent for transaction 1942 (prob=0.89)
✅ Alert sent for transaction 2591 (prob=1.00)
✅ Alert sent for transaction 2592 (prob=1.00)
✅ Alert sent for transaction 2593 (prob=1.00)
✅ Alert sent for transaction 2594 (prob=1.00)
✅ Alert sent for transaction 2595 (prob=1.00)
✅ Alert sent for transaction 2596 (prob=1.00)
✅ Alert sent for transaction 2597 (prob=1.00)
✅ Alert sent for transaction 2598 (prob=1.00)
✅ Alert sent for transaction 2599 (prob=1.00)
✅ Alert sent for transaction 2600 (prob=0.99)
✅ Alert sent for transaction 2601 (prob=1.00)
✅ Alert sent for transaction 2602 

KeyboardInterrupt: 